# <div align="center">Project 2c: Side-Channel Leakage Assessment<div>

<p><div align="center">Due Date: Sunday April 11th, 2021, 11:59 PM</div></p>

<div align="center"><b>You are expected to work in groups of two. Address all questions listed in this document in your final report. Submit the report on <a href="https://mymasonportal.gmu.edu" target="_blank">MyMason</a> in PDF format.</div>

## Test Vector Leakage Assessment
Unprotected cryptographic implementations leak information if not protected properly using side channel counter measures. To test countermeasure effectiveness, key recovery attacks (e.g DPA) can be mounted at several attack points to see if the key can be recovered. 
However, mounting such attacks, which includes analyzing the algorithm to find all possible points of attack, coming up with the power models, obtaining the measurements, etc., is time consuming and requires experience with these attacks and detailed knowledge of the implementation. You have been through this for one possible point of attack of an AES implementation in the previous parts of Project 2.
Leakage assessment methodologies allow you to quickly determine if there is a significat leakage from the implementation.

### Welch's T-Test
Welch's t-test is a moments-based statistical test used in a wide range of scientific research to show if two populations are significantly different. This test is used in the **Test Vector Leakage Assessment (TVLA)** methodology which has been used in many publications to test if there is significant information leakage from an implementation.  
If an implementation is secure against DPA, its power consumption must be independent of the algorithm's intermediate values. This implies that power traces collected when processing fixed data and traces collected when processing random data should be statistically indistinguishable. We call the two trace sets $Q_f$ and $Q_r$ respectively. A $t$ value is calculated as follows:  

\begin{equation}
t = \frac{\mu_f - \mu_r}{\sqrt{\frac{s_f^2}{n_f} + \frac{s_r^2}{n_r}}}     
\end{equation}

Where $\mu_f$ and $\mu_r$ are the means, $s_f$ and $s_r$ are the standard deviations and $n_f$ and $n_r$ are the number of samples in the sets. 
The **null hypothesis** is that the means of the two trace sets $Q_f$ and $Q_r$ are equal (i.e.,  the two trace sets are indistinguishable). We use the calculated $t$ value as an indicator to reject the null hypothesis at certain confidence level. If $\mid t \mid > 4.5 $ we reject the null hypothesis at a confidence level of $99.999\%$ (i.e., $p<10^{-5}$). This means the two sets $Q_f$ and $Q_r$ are distinguishable and the cryptographic core is likely leaking information. However, this doesn't prove that the leakage can be exploited to obtain secret information, and it doesn't recover any secret information itself.

The point of doing such a test is to give the countermeasure designer more confidence on the soundness of the protection. However, recent research has shown that even implementations passing the t-test can leak information in ways the t-test does not capture. More advanced test have been therfore developed. Never the less, TVLA is still a powerful tool and the first test to be performed for SCA leakage evaluation.

## Leakage Analysis Process

The Leakage analysis process is much simpler and shorter than the side channel analysis process. It consists of only two steps.

### Step 1: Data Acquisition - Power Measurements

We measure the power consumption of the crypto device (DUT) while it encrypts $n_f$ data blocks with the same fixed value and $n_r$ data blocks with random values. The total number of data blocks is $D=n_f+n_r$. These values can be written as a vector $\mathbf{d} = [d_1, d_2, \ldots, d_D]$. The fixed and random data blocks are randomly interleaved as the power consumption of processing the current data block might have some dependencies on the data of the previous data block. One data block i.e., plain text of the same size as the cryptographic algorithms input size in the format that the hardware implementation requires, is called a test vector. 

A single power measurement value obtained from an oscilloscope or ADC is called a sample.
All samples collected during the encryption or decryption of a single data block comprise a trace.
The trace generated while encrypting or decrypting a data block $d_i$ consists of $T$ samples and can be viewed as a vector $\mathbf{t}_i = [t_{i,1}, t_{i,2}, \ldots, t_{i,T}]$.
The traces are stacked in a matrix $\mathbf{T}$ with dimensions $D \times T$ where each row $i$ is a trace generated while encrypting or decrypting block $d_i$.


### Step 2: Data Analysis - Test Vector Leakage Assessment (TVLA)

The collected traces $\mathbf{T}$ will be split into two trace sets, one, $Q_f$ comprised of all traces recorded when the fixed data blocks were encrypted and the other one, $Q_r$ with all the traces recorded when random data blocks were encrypted. The Welche's T-test is applied to see if we can distinguish between the two sets.

# TVLA on Unprotected AES

## FOBOS Acquisition Configuration

The block of python code below this cell loads all libraries FOBOS requires and configures FOBOS for data acquisition. We now examine each section of the code.
The following steps are the same as the ones you exercized for project 2a. Only the differences are highlighted here.
 
### Files and Locations:

In order to perform TVLA, we have to generate specific test vectors that randomly interleave a fixed test vector with randomly generated ones. Therefore we give this project a new **projectName** which is also the name of the directory inside the workspace in which FOBOS stores all files pertaining to this project. As we are perfoming TVLA of an implementation of AES, we simply name it <code>aes_tvla</code>. You can change it though to anything you like as long as its a valid Linux directory name. 

FOBOS expects the following file inside the project directory:

- **bitFile**   <code>half_duplex_dut.bit</code> contains the bitstream needed to program the DUT FPGA with the algorithm under test.

Before we can run TVLA, we have to generate test vectors that the FOBOS DUT understands. We will use a script that requires a key and a fixed plaintext. It generates random plaintext, interleaves the fixed and the random plaintext in a random fashion and creates test vectors.
It keeps track of which test vector is fixed and which is random using the <code>fvrFile</code> in which is records a '0' if the plaintext was fixed and a '1' if the plaintext was random. 

- **fixedPlaintext** the value of the fixed plaintext block for AES.
- **fixedKey** the value of the key we want to use.
- **fvrFile**   <code>fvrchoicefile.txt</code> keeps track of which test vector was fixed '0' or random '1'
- **dinFile**   <code>dinFile.txt</code> contains test vectors in FOBOS format that will be send to the DUT. These testvectors contain the plaintext as well as the key. 

FOBOS creates a directory called <code>capture</code> inside the project folder. Each time you run test vectors and measure traces, a new folder <code>attempt-</code>*number* is created inside the <code>capture</code> directory. It will contain a copy of the <code>dinFile.txt</code> and the <code>fvrchoicefile.txt</code> files as well as the output files:

- **cipherFile** <code>ciphertext.txt</code> contains the ciphertext that the cipher generated from the test vectors.
- **traceFile**  <code>powerTraces.npy</code> contains the ADC / oscilloscope measurements for all traces in numpy format.

### Cipher Specific Information

Last, FOBOS needs to know some information about the cipher we want to attack, namely how big its input block size, output block size, and key size are. All numbers are to be given in bytes. This time all values are preconfigured.

Run the following cell (Shift + Enter) to import libraries needed for the capture software and configure input and output files.


In [ ]:
# import libraries
import os
import sys
import shutil
import json
import numpy as np
import matplotlib.pyplot as plt
import foboslib.traceset as traceset 
from foboslib.fvrTVGen import FvrTVGen
import foboslib.ttest
import foboslib.pynqctrl
from foboslib.fobosctrl import FOBOSCtrl
import foboslib.projmgr as projmgr
import foboslib.nexys3dut
#######################################################
# Acquisition Configuration
acqConf = {}
# Connection to Control Board
acqConf['ip'] = '192.168.10.99'
acqConf['port'] = 9995
# File Names and Locations
acqConf['workspace'] = "fobosworkspace"     # Folder to store projects
acqConf['projectName'] = "aes_tvla"         # Name of this project. Files will be in workspace/projectName/
acqConf['bitFile'] = "half_duplex_dut.bit"  # Bitstream for programming the DUT (target) 
acqConf['fixedPlaintext'] = "55555555555555555555555555555555"
acqConf['fixedKey'] = "00112233445566778899AABBCCDDEEFF"
acqConf['fvrFile'] = "fvrchoicefile.txt"    # fixed vs. random indication
acqConf['dinFile'] = "dinFile.txt"          # test vectors in FOBOS format incl. plaintext and key
acqConf['cipherFile'] = "ciphertext.txt"    # observed ciphertext, result from applying test vectors
acqConf['traceFile']  = "powerTraces.npy"   # Name of file for recording power traces
# Cipher Specific Information
acqConf['blockSize'] = 16                   # size of a block of plaintext in bytes
acqConf['cipherSize'] = 16                  # size of a block of ciphertext in bytes
acqConf['keySize'] = 16                     # size of the key in bytes


### Acquisition Settings:

The next thing we have to configure are the specifics for the test. This includes the number of traces we want to run, how fast the DUT clock should run, the oscilloscope configuration, and what the output length of the cipher is.

- **traceNum** is the number of traces we want to run. Lets start with a small number, e.g. 10.
- **DUTclk** is the clock frequency of the device under test in MHz. Slow clock frequencies lead to a cleaner signal, hence we typically run at 1 MHz.
- **samplingFreq** specifies how fast the oscilloscope should sample (measure) the power consumption. The unit is Msps which stands for Million samples per second.
- **samplesPerTrace** tells the oscilloscope after how many samples it should stop recording for each trace.
- **ADCgain** specifies the how much the signal from the power probe should be amplified. The goal is to have a signal use most of the range between 0..1023 as our ADC has a 10-bit resolution. If the sginal gets clipped, i.e. has straight lines on top at 1023 or bottom at 0, the gain was too high and has to be reduced. Please be careful when changing this parameter.


<div class="alert alert-block alert-info">

#### Questions:

1. In the code below, set the DUT clock frequency **DUTclk** to 1 MHz and oscilloscope sampling frequency **samplingFreq** to 5 Msps. How many samples will be collected every DUT clock cycle?
2. If you want to collect the entire AES encryption process for one test vector (one block), what is the number of samples you need to collect. (Hint : This implemetation needs first 4 clock cycles to load data + 11 clock cycles to finish the AES encryption). Using the number of samples you calculated, set the **samplesPerTrace** in the script. Leave all the other parameters at the default settings. Note: This time the sampling frequency is lower than in project 2a.
</div>

After you finish the configuration, run the following cell.

In [ ]:
#Acquistion configuration
acqConf['traceNum'] = 10             # number of traces to run [default: 10]
acqConf['DUTClk'] =                  # clock frequency of the DUT in MHz, [range: 1 - 100]
acqConf['samplingFreq'] =            # sampling frequency of the Oscilloscope in Msps, [range: 1 - 100]
acqConf['samplesPerTrace'] =         # number of sample in one trace [range: 1 - 2^17]
acqConf['ADCGain'] = 40              # amplification of ADC input signal [default: 40][range: 0 - 60]
# Configure project directories
pm = projmgr.ProjectManager()
pm.setWorkSpaceDir(acqConf['workspace'])
pm.setProjName(acqConf['projectName'])
projDir = pm.getProjDir()

### Generating Test Vectors

The following commands generate a file with as many plaintext blocks containing random values as we want to record traces. Approximately half of them will have the fixed value <code>fixedPlaintext</code> and the other half will have random values. It then uses them and the key to generate test vectors in FOBOS format and stores them in the file <code>dinFile</code>. It stores information about which test vector was fixed and which random in the file <code>fvrFile</code>.
In order to examine the contents of both files, traverse the directory tree on the left to *fobosworkspace/aes_tvla* and double-click the file. This will open a new tab with the contents of this file.

In [ ]:
# Generating Test Vectors
tvGen = FvrTVGen(traceNum=acqConf['traceNum'],
             blockSize=acqConf['blockSize'],
             keySize=acqConf['keySize'],
             cipherSize=acqConf['cipherSize'],
             fixedKey=acqConf['fixedKey'],
             fixedPlaintext=acqConf['fixedPlaintext'],
             dinFile= os.path.join(projDir, acqConf['dinFile']),
             fvrFile= os.path.join(projDir, acqConf['fvrFile'])
             )
tvGen.gen_fvr()

<div class="alert alert-block alert-info">

#### Questions:

3.  Look at the file <code>dinFile.txt</code>. It contains the test vectors in FOBOS 
    format, one test vector per line. Copy the file into your report and determine 
    which vectors are fixed and which are randomly generated.
4.  How many fixed test vectors does <code>dinFile.txt</code> contain and how many 
    random test vectors.
5.  Look at the entries in the fvrchoicefile.txt (0's and 1's). What do they mean?  
</div>

## FOBOS Acquisition Measurements

### Run the Measurements

The next cell runs the measurements. First, it acquires access and then connects to FOBOS test setup: "hardware". This locks the hardware while you are  using it. Once you finish the hardware is released to allow others to access it.
You should see a message saying 'Acquired hardware lock'. If the hardware is used it will wait for some time until hardware is available. If this does not work, please try again after few minutes. But **please do not interrupt this cell if it is still trying to acquire access**.

Once the hardware is accessed, the script first applies the configuration parameters you configured previously to the control board and the oscilloscope / ADC. Then it programms the FPGA of the DUT. It specifies the input and output files and creates a new folder <code>attempt-</code>number inside the <code>capture</code> directory for the output files and copies the input files <code>dinFile.txt</code> and <code>fvrchoicefile.txt</code> files into it. 

<span style="color:red">Please pay attention to the messages generated by the next cell and note the attempt-xx directory number.</span>

After all this setup, it sends the test vectors one at a time to the control board, receives the ciphertext and power traces.

Once its done, it will release the hardware lock so that other students can use it. You should see a message saying 'Released hardware lock.'

If the cell ends with an error message and does not release the hardware lock, you will have to wait for 5 minutes until the lock is released automatically before you can run this cell again.

In [ ]:
# connect to hardware and lock it ######################################
ctrl = foboslib.pynqctrl.PYNQCtrl(acqConf['ip'], acqConf['port'])
# Apply config to ctrl board ###########################################
try:
    ctrl.setDUTClk(acqConf['DUTClk'])
    ctrl.setDUTInterface(FOBOSCtrl.INTERFACE_4BIT)
    ctrl.setOutLen(acqConf['cipherSize'])
    ctrl.setTriggerMode(FOBOSCtrl.TRG_FULL)
    # configure trace sampling #############################################
    ctrl.setSamplingFrequency(acqConf['samplingFreq'])
    ctrl.setADCGain(acqConf['ADCGain'])
    ctrl.setSamplesPerTrace(acqConf['samplesPerTrace'])
except Exception as e:
    print(e)
    ctrl.disconnect()
else:
    # program DUT ##########################################################
    dut = foboslib.nexys3dut.Nexys3DUT()
    bitFile = os.path.join(projDir, acqConf['bitFile'])
    dut.setBitFile(bitFile)
    dut.program()
    # prepare i/o files ####################################################
    tvFileName = os.path.join(projDir, acqConf['dinFile'])
    #plainFileName = os.path.join(projDir, acqConf['plainFile'])
    fvrFileName = os.path.join(projDir, acqConf['fvrFile'])
    tvFile = open(tvFileName, "r")
    captureDir = pm.getCaptureDir()
    cipherFileName = os.path.join(captureDir, acqConf['cipherFile'])
    cipherFile = open(cipherFileName, "w")
    traceFileName = os.path.join(captureDir, acqConf['traceFile'])
    traceFile = open(traceFileName, "a+b")
    shutil.copy(tvFileName, captureDir)
    #shutil.copy(plainFileName, captureDir)
    shutil.copy(fvrFileName, captureDir)
    # save config to a file
    configFile = open(os.path.join(captureDir, 'acquisitionConfig.json'), "w")
    configFile.write(json.dumps(acqConf, indent=4))
    #configFile.write(ctrl.config)
    #print(ctrl.config)
    configFile.close()
    # Get traces############################################################
    print('Processing test vectors ...')
    traceNum = 0
    while traceNum < acqConf['traceNum']:
        data = tvFile.readline()
        status, result, trace = ctrl.processData2(data, acqConf['cipherSize'])
        cipherFile.write(result + "\n")
        np.save(traceFile, trace)
        if traceNum % 100 == 0:
            sys.stdout.write('Progress:' + "{:.2f}".format(traceNum/acqConf['traceNum']*100) + '%\r')
            sys.stdout.flush()
        traceNum += 1
    print('Data acquisition complete.')
    ctrl.disconnect() # release and reset control board.
    traceFile.close()
    cipherFile.close()
    tvFile.close()

### Plot and Evaluate the Measurements
The next code cell will create a graph of the measured power consumption and plot up to 100 traces on top of each other. You can adjust the size of the plot by changing the entry for <code>figsize</code>. The figure will be saved under the name <code>traces.png</code> in the <code>capture/attempt-xx</code> directory.

In [ ]:
fig = plt.figure()
fig.patch.set_facecolor('white')
plt.rcParams.update({'font.size': 18})
traceFile = open(traceFileName, "r+b")
maxtrace = 100
plt.figure(figsize=(10,8))
plt.xlabel('Sample')
plt.ylabel('Amplitude')
plt.title('Captured Traces')
for i in range(min(maxtrace, acqConf['traceNum'])):
    trace = np.load(traceFile)
    plt.plot(trace)

plt.savefig(os.path.join(captureDir, 'traces.png'),facecolor=fig.get_facecolor())
# plt.close()
traceFile.close()

<div class="alert alert-block alert-info">

#### Questions:

6.  What files does the *capture/attempt-xx* directory contain?
7.  Now lets examine the graph. Do you see all 11 peaks of AES similar to 
    project 2a? If not, adjust the number of **samplesPerTrace** and re-run 
    the boxes above starting from <code>#Acquisition Settings</code> until 
    you see the wave form for the complete AES. 
    Write the settings you used into your report.
8.  Copy the generated graph into your report and mark the peaks relating 
    to AES indicating the round numbers.
9.  Even though you see now all 11 peaks of AES, why does the graph looks different 
    than the one from project 2a.
10. Which range of samples corresponds to the execution of the AES algorithm? 
    Do not include samples during transmission of data to and from AES.
</div>

## FOBOS Analysis

### FOBOS Analysis Setup

FOBOS Analysis needs to know the directory in which the measurements are stored and the accompanying acquisition settings. 

* **WORKSPACE1** is the location of the <code>capture</code> directory that got created for the acquisition step as **workspace**/**projectName**/capture. Inside this directory, each time an acquisition was performed an **ATTEMPT_DIR** was created, named <code>attempt-</code>*number*. 
* **ATTEMPT_DIR1** contains the measurements, ciphertext, and the acquisition settings.

Set the values accordingly in the cell below and execute it.

In [ ]:
# Configure project directories 
WORKSPACE1 = "fobosworkspace/aes_tvla/capture"   # Workspace for FOBOS Analysis
ATTEMPT_DIR1 = "attempt-xx"                      # the location is likely in the form attampt-i (e.g. attempt-01)
###########################
pm = projmgr.ProjectManager()
pm.setWorkSpaceDir(WORKSPACE1)
pm.setProjName(ATTEMPT_DIR1)
projDir1 = pm.getProjDir()
analysisDir1 = pm.getAnalysisDir()

<span style="color:red">Please pay attention to the messages generated by the previous cell and note the attempt-xx directory number.</span>

#### Files and Locations:

Now we can load the acquistion settings. The locations of the file that contains the traces and the file that kept track of which traces are fixed and which are random are retrieved from the acquisition configuration.

In [ ]:
# Load Acquisition Config
tvlaConf = {}
configFile = open(os.path.join(projDir1, 'acquisitionConfig.json'))
acqConf = json.load(configFile)
print(f'Acquisition config = {acqConf}')
tvlaConf['traceNum'] = acqConf['traceNum']

# Configure file names
TRACES_FILE1 = os.path.join(projDir1, acqConf['traceFile'])  # Name of file for recording power traces
FVR_FILE1 = os.path.join(projDir1, acqConf['fvrFile'])

Run the following cell to display some of the traces collected form FOBOS running the unproteced AES. The graph should be exactly the same as you saw above. If this cell fails, then most likely you picked the wrong <code>attempt-xx</code>.

In [ ]:
##t-test/plotting settings
tvlaConf['plot'] = True
tvlaConf['plotSize'] = (10,8)
tvlaConf['plotFontSize'] = 18
MAX_TRACES = 100
max_traces = min(MAX_TRACES, tvlaConf['traceNum'])
# load traces from file.
traceSetOrig = traceset.TraceSet(traceNum=max_traces,
                                 fileName=TRACES_FILE1)
measuredPowerOrig = traceSetOrig.traces
print(f'The shape of the traces matrix is {measuredPowerOrig.shape}')
fig = plt.figure()
fig.patch.set_facecolor('white')
plt.figure(figsize=tvlaConf['plotSize'])
plt.rcParams.update({'font.size': tvlaConf['plotFontSize']})
plt.xlabel('Sample')
plt.ylabel('Amplitdue')
plt.title('Traces')
for i in range(max_traces):
    plt.plot(measuredPowerOrig[i])
    
plt.savefig(os.path.join(analysisDir1, 'traces.png'),facecolor=fig.get_facecolor())

### Splitting Traces

The collected traces $\mathbf{T}$ will have to be split into two trace sets, one, $Q_f$ comprised of all traces recorded when the fixed data blocks were encrypted and the other one, $Q_r$ with all the traces recorded when random data blocks were encrypted. 
The code below looks at <code>fvrFile</code> to determine which trace resulted from a test vector was fixed or random and stores the traces from the fixed test vectors in <code>trace_set0.npy</code> and from the random test vectors in <code>trace_set1.npy</code>. This might take some time when you have a large amount of test vectors.

In [ ]:
# load power traces
fvrtraces = traceset.TraceSet(traceNum=tvlaConf['traceNum'],
                              fileName=TRACES_FILE1)
# Split them into fixed traces and random traces
test = foboslib.ttest.Ttest()
test.split(traces=fvrtraces.traces, 
          fvrFile=FVR_FILE1,
          numTraces=tvlaConf['traceNum'],
          analysisDir=projDir1)

<div class="alert alert-block alert-info">

#### Questions:

11. Look at the output of the previous code cell. 
    How many traces were taken for the fixed test vectors and how many for the 
    random test vectors. Does this match your result from question 4?
</div>

## Test Vector Leakage Assessment

Run the following code to perform the t-test. It will use the two power trace files generated in the previous step.

The plot has sample numbers on the x-axis and t-values on the y-axis.
Two lines at 4.5 and -4.5 indicate the threshold discussed above.

The **step** parameter in the code below allows the input files to be loaded in small chunks to avoid filling the memory.

In [ ]:
traceFile0 = os.path.join(projDir1, 'trace_set0.npy')
traceFile1 = os.path.join(projDir1, 'trace_set1.npy')

tvals = test.doTtest(traceFile0=traceFile0,
                  traceFile1=traceFile1, 
                  numTraces=tvlaConf['traceNum'],
                  step='auto',
                  analysisDir=analysisDir1)
test.plotTValues(t=tvals, 
                 startXlim=0, 
                 endXlim='All', 
                 startYlim=-50, 
                 endYlim=50, 
                 analysisDir=analysisDir1,
                 traceNum=tvlaConf['traceNum'],
                 plotSize=tvlaConf['plotSize'],
                 plotFontSize=tvlaConf['plotFontSize']
                )

<div class="alert alert-block alert-info">

#### Questions:
12. Include the graph above for 10 traces in your report. 
    It has been stored in <code>analysis/attempt-</code>*number* directory and 
    is called <code>t-test-result-10-traces.png</code>.
13. What do you observe about the result, does this implementation about AES pass 
    the t-test?
14. Now set the trace number to 100 in the Acquisition Settings and run the 
    the cells again. Don't forget the change the <code>capture/attempt-</code>
    in the FOBOS Analysis setup step.
    Include the resulting graph in your report.
15. Now set the trace number to 1000 and run the code again. 
    Include the resulting graph in your report.
16. What differences do you observe in the three graphs?
17. Compare the last graph to the power traces you plotted above. 
    Based on the sample number, can you identify during which AES clock cycles 
    information is leaking?
</div>

## The effect of sampling frequency

In this section, we will increase the sampling rate back to 50 Msps which is the same frequency we used in project 2a. 




<div class="alert alert-block alert-info">

#### Questions:
18. Go back to the Acquisition Settings and set the oscilloscope sampling frequency 
    **samplingFreq** to 50 Msps. How many samples will be collected every DUT clock
    cycle?
19. Adjust the **samplesPerTrace** so that a full AES encryption is captured 
    with the new sampling frequency. Write the new value in your report.
20. Set the number of traces back to 10 and run the code again. Include the 
    resulting graph in your report.
21. Repeat step 20 this for 100 and 1000 traces.
22. What differences do you observe between the graphs generated with 
    the two different sampling rates of 5 Msps and 50 Msps.
    (Note: Compare t-tests with the same number of traces)
23. Compare Correlation Power Analysis (project 2a and 2b) 
    to Test Vector Leakage Assessment (project 2c) in terms of 
    - how much time you spent,
    - how sensitive a successful attack is to the choice of parameters for 
      data acquisition, and
    - coverage of propable leakage points.
</div>

The last questions are about your experience with this project.
<div class="alert alert-block alert-info">

#### Questions:
24. How much time did you spend on project 2c.
25. What concept was the hardest to understand.
26. Which step took the most time.
27. What were the biggest hurdles you had to overcome.
28. How could this project be improved.
</div>